In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession.\
    builder.\
    config('spark.ui.port','0').\
    master('yarn').\
    getOrCreate()

In [2]:
## To store a single large file in to multiple floders on expense_type 

In [3]:
from pyspark.sql.types import *

In [4]:
transactions_schema = StructType([
	StructField("cus_id",StringType()),	
	StructField("start_date",DateType()),
	StructField("end_date",DateType()),
	StructField("trans_id",StringType()),
	StructField("date_of_transaction",DateType()),
	StructField("year",StringType()),
	StructField("month",StringType()),
	StructField("day",StringType()),
	StructField("exp_type",StringType()),
	StructField("amount",FloatType()),
])

In [5]:
transactions_df = spark.read \
.format("csv") \
.option("InferSchema","True") \
.schema(transactions_schema) \
.option("Header","True") \
.load("/user/itv007180/transactions.csv")

In [6]:
## Below is the structure of the file transactions.csv

In [7]:
transactions_df.show(5)

+----------+----------+--------+---------------+-------------------+----+-----+---+-------------+------+
|    cus_id|start_date|end_date|       trans_id|date_of_transaction|year|month|day|     exp_type|amount|
+----------+----------+--------+---------------+-------------------+----+-----+---+-------------+------+
|CI6XLYUMQK|2015-05-01|    null|T8I9ZB5A6X90UG8|         2015-09-11|2015|    9| 11| Motor/Travel| 20.27|
|CI6XLYUMQK|2015-05-01|    null|TZ4JSLS7SC7FO9H|         2017-02-08|2017|    2|  8| Motor/Travel| 12.85|
|CI6XLYUMQK|2015-05-01|    null|TTUKRDDJ6B6F42H|         2015-08-01|2015|    8|  1|      Housing| 383.8|
|CI6XLYUMQK|2015-05-01|    null|TDUHFRUKGPPI6HD|         2019-03-16|2019|    3| 16|Entertainment|  5.72|
|CI6XLYUMQK|2015-05-01|    null|T0JBZHBMSVRFMMD|         2015-05-15|2015|    5| 15|Entertainment| 11.06|
+----------+----------+--------+---------------+-------------------+----+-----+---+-------------+------+
only showing top 5 rows



In [8]:
## creating a new df by eliminating the unecessary fields from original df

In [9]:
transactions_new_df = transactions_df.select("cus_id","date_of_transaction","exp_type","amount")

In [10]:
## Extracting only the active clients by eliminating the customers who already has end_date populated 

In [11]:
transactions_new_df1 = transactions_df.filter("end_date is NULL").select("cus_id","date_of_transaction","exp_type","amount")

In [12]:
transactions_new_df1.count()

5578907

In [13]:
## Writing to HDFS by using write api and partitionBy on exp_type

In [17]:
transactions_new_df1.write \
.format("csv") \
.option("mode","overwrite") \
.partitionBy("exp_type") \
.option("path","/user/itv007180/trans4").save()